In [23]:
import pandas as pd
import numpy as np
import os

import seaborn as sns

from matplotlib import pyplot as plt

In [2]:
path_to_data = '../../../_datasets/sas'

In [3]:
train = pd.read_csv(os.path.join(path_to_data, 'train.csv'), encoding='cp1251')

/home/mikhail/venv/mlprojects/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Preprocessing

In [95]:
order_cols = ['OrderID', 'Date', 'OrderDate', 'ChannelID', 'ClientID', 'DeliveryType', 'count_edit', 
              'prepay', 'Interval', 'Cluster', 'CancelFlag']

orders = train.drop_duplicates(subset=['OrderID'], keep='first')[order_cols].reset_index(drop=True)

In [96]:
#orders = orders[orders.ClientID ==93925152]

In [97]:
orders['DeliveryType'] = orders['DeliveryType'].map({'Доставка День в День':0, 'Обычная доставка':1})

In [98]:
orders[['Date', 'OrderDate']] = orders[['Date', 'OrderDate']].apply(pd.to_datetime)

### Exploratory Data Analysis

In [ ]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values
    summary['First Value'] = df.loc[0].values
    summary['Second Value'] = df.loc[1].values
    summary['Third Value'] = df.loc[2].values

    #for name in summary['Name'].value_counts().index:
    #    summary.loc[summary['Name'] == name, 'Entropy'] = round(stats.entropy(df[name].value_counts(normalize=True), base=2),2) 

    return summary


def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
orders = reduce_mem_usage(orders)

In [ ]:
summary = resumetable(orders)
summary

In [ ]:
orders.info()

In [ ]:
orders.isnull().any()

In [ ]:
orders.dtypes

**1.1 Binary columns**

In [ ]:
bin_cols = ['prepay', 'DeliveryType']

In [ ]:
plt.figure(figsize=(15,4))

for i, col in enumerate(orders[bin_cols]):
    ax = plt.subplot(1, 2, i+1)
    sns.countplot(x=col, data=orders, hue='CancelFlag', palette='hls')
    ax.set_ylabel('Count', fontsize=8) # y axis label
    ax.set_title(f'{col} Distribution by CancelFlag', fontsize=10) # title label
    ax.set_xlabel(f'{col} values', fontsize=8) # x axis label

    
    total = len(orders)

    sizes=[] # Get highest values in y

    for p in ax.patches: # loop to all objects
        height = p.get_height()
        sizes.append(height)

        ax.text(p.get_x()+p.get_width()/2.,
                height + 3,
                '{:1.2f}%'.format(height/total*100),
                ha="center", fontsize=8) 

**1.2 Nominal Features (with more than 2 and less than 15 values)**

In [ ]:
nom_cols = ['Interval', 'Cluster', 'ChannelID']

In [ ]:
plt.figure(figsize=(20,20))

for i, col in enumerate(nom_cols):
    tmp = pd.crosstab(train[col], train['CancelFlag'], normalize='index') * 100
    tmp = tmp.reset_index()
    tmp.rename(columns={0:'No',1:'Yes'}, inplace=True)
    
    ax = plt.subplot(4, 1, i+1)
    sns.countplot(x=col, data=train, order=list(tmp[col].values) , color='green')
    ax.set_ylabel('Count', fontsize=8) # y axis label
    ax.set_title(f'{col} Distribution by CancelFlag', fontsize=10) # title label
    ax.set_xlabel(f'{col} values', fontsize=8) # x axis label

    # Target %True
    gt = ax.twinx()
    gt = sns.pointplot(x=col, y='Yes', data=tmp,
                       order=list(tmp[col].values),
                       color='black', legend=False)

### Feature Engineering

In [99]:
orders = orders.sort_values(by=['OrderDate'], ascending=True)
orders.head()

,OrderID,Date,OrderDate,ChannelID,ClientID,DeliveryType,count_edit,prepay,Interval,Cluster,CancelFlag
17068,98354363910,2018-01-20,2018-01-19,2,93925152,1,1,0,6-8.,NaN,0
30586,98354473406,2018-02-02,2018-01-31,2,93925152,1,1,0,6-8.,NaN,0
58745,98354726604,2018-02-27,2018-02-26,2,93925152,1,1,0,6-8.,NaN,0
87947,98354967809,2018-03-25,2018-03-24,2,93925152,1,1,0,10-12.,NaN,0
120258,98355218491,2018-04-25,2018-04-24,2,93925152,1,1,0,6-8.,NaN,0


**<span style="color:red">1. Datetime features</span>**

In [108]:
orders = orders.assign(year=orders.OrderDate.dt.year,
                       month=orders.OrderDate.dt.month,
                       day=orders.OrderDate.dt.day,
                       weekday=orders.OrderDate.dt.dayofweek)

**<span style="color:red">2. Number of orders in the past 1-day, 7-days</span>**

In [109]:
def count_past_events(series, time_window='60D'):
    series = pd.Series(series.index, index=series)
    # Subtract 1 so the current event isn't counted
    past_events = series.rolling(time_window).count() - 1
    return past_events

In [110]:
#for i in [1, 3, 5, 10, 20, 60]:
for i in [60]:
    days = '{}D'.format(i)
    number_of_orders = orders.groupby('ClientID')['OrderDate'].transform(count_past_events, time_window=days)
    
    orders['number_of_orders_past_{}_days'.format(i)] = number_of_orders

**<span style="color:red">3. Days since last order</span>**

In [111]:
def time_diff(series):
    # время в днях
    return series.diff().dt.total_seconds()/60/60/24

In [112]:
timedeltas = orders.groupby('ClientID')['OrderDate'].transform(time_diff)

orders['days_since_last_order'] = timedeltas

**<span style="color:red">4. Number of previous cancelled orders</span>**

In [113]:
def previous_attributions(series):
    # Subtracting raw values so I don't count the current event
    sums = series.expanding(min_periods=2).sum() - series
    return sums

In [114]:
orders_count = orders.groupby('ClientID')['CancelFlag'].transform(previous_attributions)

orders['number_of_previous_canselled_order'] = orders_count

In [115]:
orders

,OrderID,Date,OrderDate,ChannelID,ClientID,DeliveryType,count_edit,prepay,Interval,Cluster,CancelFlag,year,month,day,weekday,number_of_orders_past_60_days,time_since_last_order,number_of_previous_canselled_order,days_since_last_order
17068,98354363910,2018-01-20,2018-01-19,2,93925152,1,1,0,6-8.,NaN,0,2018,1,19,4,0.0,NaN,NaN,NaN
30586,98354473406,2018-02-02,2018-01-31,2,93925152,1,1,0,6-8.,NaN,0,2018,1,31,2,1.0,12.0,0.0,12.0
58745,98354726604,2018-02-27,2018-02-26,2,93925152,1,1,0,6-8.,NaN,0,2018,2,26,0,2.0,26.0,0.0,26.0
87947,98354967809,2018-03-25,2018-03-24,2,93925152,1,1,0,10-12.,NaN,0,2018,3,24,5,2.0,26.0,0.0,26.0
120258,98355218491,2018-04-25,2018-04-24,2,93925152,1,1,0,6-8.,NaN,0,2018,4,24,1,2.0,31.0,0.0,31.0
140103,98355375395,2018-05-17,2018-05-16,2,93925152,1,1,0,10-12.,NaN,0,2018,5,16,2,2.0,22.0,0.0,22.0
148851,98355444159,2018-05-26,2018-05-25,2,93925152,1,1,0,8-10.,NaN,0,2018,5,25,4,2.0,9.0,0.0,9.0
159100,98355524366,2018-08-06,2018-06-06,2,93925152,1,1,0,6-8.,NaN,0,2018,6,6,2,3.0,12.0,0.0,12.0
184431,98355727228,2018-08-07,2018-06-07,2,93925152,1,1,0,6-8.,NaN,0,2018,6,7,3,4.0,1.0,0.0,1.0
208561,98355936320,2018-07-08,2018-06-08,2,93925152,1,1,0,16-18.,NaN,0,2018,6,8,4,5.0,1.0,0.0,1.0
